In [34]:
import sys
sys.path.insert(0, "/home/krzysiek/Development/solvency-modelling/solvency_models/notebooks/utils")

from utils.my_kedro import MyKedro, suppress_all_loggers
import logging

logger = logging.getLogger(__name__)
logger.setLevel("INFO")

my_kedro = MyKedro()
# my_kedro = MyKedro(mlflow_run_id="332778368c374987b6a03b81e96851a9")
config = my_kedro.run("init", level="ERROR")["config"]

In [41]:
my_kedro.reload()
config = my_kedro.run("init", level="ERROR")["config"]

In [2]:
task_out = my_kedro.run("all_to_smpl")

In [35]:
task_out = my_kedro.run("de", level="DEBUG")

In [36]:
task_out = my_kedro.run(pipeline_name="ds")

In [40]:
task_out = my_kedro.run(pipeline_name="clb")

In [42]:
task_out = my_kedro.run(pipeline_name="test")

In [47]:
catalog.list()

In [65]:
calibrated_calib_predictions_df = catalog.load("calibrated_test_predictions_df")

In [66]:
calibrated_calib_predictions_df.head()

In [64]:
task_out = my_kedro.run(pipeline_name="test", level="DEBUG")

In [5]:
task_out = my_kedro.run(pipeline_name="ds", node_names=["process_parameters", "select_features"])

In [30]:
task_out = my_kedro.run(pipeline_name="ds", node_names=["process_parameters", "fit_predictive_model"])

In [74]:
# catalog.list()

In [301]:
sample_features_df = catalog.load("sample_features_df")

In [214]:
sample_features_df.head()

In [5]:
transformed_sample_features_df = catalog.load("transformed_sample_features_df")
sample_target_df = catalog.load("sample_target_df")

In [216]:
transformed_sample_features_df.head()

In [57]:
rfe.get_selected_features()

In [60]:
transformed_sample_features_df.loc[:,rfe._selected_features]

In [217]:
transformed_sample_features_df.describe()

In [218]:
sample_target_df.head()

In [219]:
config.mdl_task.target_col

In [220]:
config.mdl_task.prediction_col

In [43]:
rfe = RecursiveFeatureEliminationSelector(config)
rfe.fit(transformed_sample_features_df, sample_target_df)
rfe.get_features_importances()

In [40]:
importances = rfe.selector.estimator_.get_features_importances().copy()
importances.index = rfe.selector.feature_names_in_[importances.index]
importances

In [30]:
rfe.selector.feature_names_in_

In [46]:
"\n".join(rfe.get_selected_features().tolist())

In [29]:
rfe.selector.feature_names_in_ rfe.selector.estimator_.get_features_importances()

In [15]:
rfe.get_features_importances()

In [24]:
rfe.get_selected_features()

In [20]:
pd.Series(1 / rfe.selector.ranking_, index=pd.Index(rfe.selector.feature_names_in_))

In [401]:
# from solvency_models.pipelines.p07_data_science.models.glm import GeneralizedLinearModel
glm = GeneralizedLinearModel(config)
glm.fit(transformed_sample_features_df, sample_target_df)
glm.get_features_importances()

In [363]:
rfe.selector.feature_names_in_

In [365]:
rfe.selector.support_

In [364]:
rfe.selector.feature_names_in_[rfe.selector.support_]

In [312]:
lasso.get_selected_features()

In [176]:
model = GeneralizedLinearModel(config)

In [177]:
model.fit(transformed_sample_features_df, sample_target_df)

In [178]:
prediction_df = model.predict(transformed_sample_features_df)

In [173]:
s = pd.Series([0, 1, 2, 3, 4])
pd.DataFrame(s)

In [179]:
prediction_df

In [127]:
prediction_df.name

In [183]:
model.summary()

In [92]:
prediction_df

In [25]:
from solvency_models.pipelines.utils.mlflow_model import MLFlowModelLoader
from solvency_models.pipelines.p08_model_calibration.utils import _calibration_model_artifact_path

clb_model = MLFlowModelLoader("calibration model").load_model(path=_calibration_model_artifact_path,
                                                                  run_id=config.clb.mlflow_run_id)

In [26]:
clb_model

In [27]:
clb_model.y_thresholds_

In [28]:
clb_model.X_thresholds_

In [31]:
import numpy as np

# Assuming clb_model.y_thresholds_ is a numpy array
y_thresholds = clb_model.y_thresholds_

# Define the x coordinates for the data points (xp)
xp = np.arange(len(y_thresholds))

# Define the x coordinates at which to evaluate the interpolant (x)
x = np.linspace(0, len(y_thresholds) - 1, num=100)

# Perform linear interpolation
interpolated_y_thresholds = np.interp(x, xp, y_thresholds)

In [32]:
def calibrate(clb_model, y):
    return np.interp(y, clb_model.X_thresholds_, clb_model.y_thresholds_)

In [33]:
y_cal = calibrate(clb_model, calibrated_test_predictions_df.pure_ClaimFreq_PRED)

In [34]:
len(y_cal)

In [35]:
y_cal

In [36]:
calibrated_test_predictions_df.pure_ClaimFreq_PRED

In [37]:
len(np.unique(y_cal))

In [38]:
len(np.unique(calibrated_test_predictions_df.pure_ClaimFreq_PRED))

In [39]:
len(np.unique(calibrated_test_predictions_df.calibrated_ClaimFreq_PRED))

In [42]:
y_pure_clb = calibrate(clb_model, calibrated_test_predictions_df.calibrated_ClaimFreq_PRED)

In [10]:
target_df = catalog.load("target_df")

In [9]:
calibrated_calib_predictions_df = catalog.load("calibrated_calib_predictions_df")

In [65]:
len(np.unique(y_cal))

In [11]:
calibrated_calib_predictions_df

In [12]:
target_df

In [75]:
pure_sample_predictions_df = catalog.load("pure_sample_predictions_df")

In [76]:
pure_sample_predictions_df

In [77]:
target_df = catalog.load("target_df")

In [78]:
target_df

In [13]:
y = target_df.loc[calibrated_calib_predictions_df.index, config.mdl_task.target_col]
x = calibrated_calib_predictions_df.pure_ClaimFreq_PRED

In [16]:
from cir_model import CenteredIsotonicRegression

In [14]:
y

In [15]:
x

In [18]:
cir_model = CenteredIsotonicRegression()
cir_model.fit(x, y)
y_cir_cal = cir_model.predict(x)
y_cir_cal

In [59]:
len(np.unique(y_cir_cal))

In [19]:
import sklearn

In [20]:
sklearn.__version__

In [22]:
import cir_model

In [23]:
cir_model.__version__

## Exploring results

In [44]:
import mlflow

In [45]:
mlflow.active_run().info

In [ ]:
from solvency_models.pipelines.p07_data_science.models.glm import GeneralizedLinearModel